In [1]:
import sys
print(sys.prefix)
from pprint import pprint

/home/jinda/anaconda3/envs/cs224n


In [2]:
# !pip install transformers[torch]
import os
import accelerate
from packaging import version

def check_and_update_accelerate():
    # Check the current version of accelerate
    current_version = version.parse(accelerate.__version__)

    # Define the required version
    required_version = version.parse("0.20.1")

    if current_version < required_version:
        # Update accelerate
        !pip install accelerate -U

        # Display a message indicating the update
        print(f"accelerate updated to version {required_version}.")
        os.kill(os.getpid(), 9)
        

    else:
        print("accelerate is already up to date.")

# Call the function to check and update accelerate
check_and_update_accelerate()


accelerate is already up to date.


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  6 23:18:30 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  | 00000000:06:00.0 Off |                  N/A |
| 44%   60C    P0              79W / 250W |     91MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install datasets jiwer

In [5]:
# ensure enough memory present so that training does not stop
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 67.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [6]:
# Import libraries
import torch
from transformers import BartTokenizerFast, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from jiwer import wer
from huggingface_hub import notebook_login

2023-12-06 23:18:35.336089: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-06 23:18:35.384303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 23:18:36.454872: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
!pip install --upgrade huggingface_hub

In [8]:
notebook_login()

map columns for noisy data

In [9]:
!pwd

/home/jinda/Desktop/torgo/tatoeba-inference/Machine Translation


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file into a DataFrame
df = pd.read_csv('data/sentences_noise_data_sample.csv')
df

,Sentence,Alternative1,Alternative2,Alternative3,Best_Alternative
0,She applied her mind to her new job.,ro sth erj oplecemdly sheab pliedh e rm imdto ...,zhe,shep uz hethi mt oqui dhisjop,Alternative3
1,Sami was pulling Layla's hair.,zpurlinkla yla zhai r z ani hidlayl aover dhe...,s amiwa,let,Alternative2
2,How could I have forgotten it?,thele h owc ourdihave vorg ottenit,t,howcour dih ave be en such ajerg,Alternative3
3,I thought both Tom and Mary were single.,o thto mandnalyw elesingre ithin,i t hou khtb,ktona ndma ry gmowa rreat y,Alternative1
4,He always goes home by himself.,lwa ystriez to havehiz ownway he always koes h...,he a,h eishisu s ualse lf,Alternative3
...,...,...,...,...,...
9995,I can't discuss it.,ica ndtis cussit,ic a mt temythad,icantm ake it,Alternative1
9996,Rima heard it.,ri mahear di t,NaN,ziliristene ttorinaze xp lanad ion,Alternative1
9997,Sami and Layla met on a Cairo subway.,san iandra yla ne tonacairo zu b way,la ylafisiteds ami i mcailo,NaN,Alternative1
9998,"According to the coach, Bob is a good swimmer.",r zw imne ldha nm aryiz acco rtingt othe coa ...,donisamuchp e tde,iswinb le tdywe lr,Alternative1


In [11]:
dataset = pd.DataFrame({
    'Sentence': df['Sentence'],
    'noisy_sentence': df.apply(lambda row: row[row['Best_Alternative']], axis=1)
})
dataset

,Sentence,noisy_sentence
0,She applied her mind to her new job.,shep uz hethi mt oqui dhisjop
1,Sami was pulling Layla's hair.,s amiwa
2,How could I have forgotten it?,howcour dih ave be en such ajerg
3,I thought both Tom and Mary were single.,o thto mandnalyw elesingre ithin
4,He always goes home by himself.,h eishisu s ualse lf
...,...,...
9995,I can't discuss it.,ica ndtis cussit
9996,Rima heard it.,ri mahear di t
9997,Sami and Layla met on a Cairo subway.,san iandra yla ne tonacairo zu b way
9998,"According to the coach, Bob is a good swimmer.",r zw imne ldha nm aryiz acco rtingt othe coa ...


In [12]:
import pandas as pd
from jiwer import wer, cer

# Assuming 'df' contains your data
# Replace NaN values with empty strings or any appropriate placeholder
df.fillna('', inplace=True)

# Create the dataset DataFrame
dataset = pd.DataFrame({
    'Sentence': df['Sentence'],
    'noisy_sentence': df.apply(lambda row: row[row['Best_Alternative']], axis=1)
})

dataset = dataset[dataset['noisy_sentence'] != '']

dataset = dataset[dataset['Sentence'] != '']

dataset

,Sentence,noisy_sentence
0,She applied her mind to her new job.,shep uz hethi mt oqui dhisjop
1,Sami was pulling Layla's hair.,s amiwa
2,How could I have forgotten it?,howcour dih ave be en such ajerg
3,I thought both Tom and Mary were single.,o thto mandnalyw elesingre ithin
4,He always goes home by himself.,h eishisu s ualse lf
...,...,...
9995,I can't discuss it.,ica ndtis cussit
9996,Rima heard it.,ri mahear di t
9997,Sami and Layla met on a Cairo subway.,san iandra yla ne tonacairo zu b way
9998,"According to the coach, Bob is a good swimmer.",r zw imne ldha nm aryiz acco rtingt othe coa ...


In [13]:
def calculate_wer(row):
    if row['Sentence'] and row['noisy_sentence']:
        return wer(row['Sentence'], row['noisy_sentence'])*100
    else:
        return -1

def calculate_cer(row):
    if row['Sentence'] and row['noisy_sentence']:
        return cer(row['Sentence'], row['noisy_sentence'])*100
    else:
        return -1

In [14]:
# Apply the WER calculation function to the dataset
dataset['WER'] = dataset.apply(calculate_wer, axis=1)
dataset['CER'] = dataset.apply(calculate_cer, axis=1)
cer_dataset = dataset[dataset['CER'] <= 50]
cer_dataset

,Sentence,noisy_sentence,WER,CER
9,Tom is probably a bit younger than I am.,tonis br obab ry abi d younk ert haniam,100.000000,45.000000
11,I'm going to freeze to death out here.,i mgoi ngtov reeze tod eath oudhere,100.000000,36.842105
19,Let's play together.,redzp la ytokether,100.000000,45.000000
20,I feel as I can rely on things now.,i ve el asic anle l yonthin kzmow,100.000000,48.571429
21,Ziri's company leased a piece of land from the...,i scom pa nyreas eda bieceovl a ndvronthe loc ...,100.000000,45.312500
...,...,...,...,...
9987,Terrace farming is widely practiced in the mou...,terr acefarmimgiswiterybl acd ic e di nth emo...,100.000000,37.500000
9990,Tom wanted to see what was in the cave.,domwa n tet to se e whatwazin thecave,100.000000,33.333333
9992,I have to stop this.,i have tozt opd his,80.000000,40.000000
9995,I can't discuss it.,ica ndtis cussit,100.000000,47.368421


In [15]:
!pip install g2p_en

In [16]:
from g2p_en import G2p

g2p = G2p()

cer_dataset['Sentence'] = cer_dataset['Sentence'].apply(lambda x: " ".join(g2p(x)))
cer_dataset['noisy_sentence'] = cer_dataset['noisy_sentence'].apply(lambda x: " ".join(g2p(x)))
cer_dataset

/tmp/ipykernel_3587526/277554369.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cer_dataset['Sentence'] = cer_dataset['Sentence'].apply(lambda x: " ".join(g2p(x)))
/tmp/ipykernel_3587526/277554369.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cer_dataset['noisy_sentence'] = cer_dataset['noisy_sentence'].apply(lambda x: " ".join(g2p(x)))


,Sentence,noisy_sentence,WER,CER
9,T AA1 M IH1 Z P R AA1 B AH0 B L IY0 AH0 ...,T OW1 N AH0 S B IY1 R OW1 B AH0 B R AY1 ...,100.000000,45.000000
11,AY1 M G OW1 IH0 NG T UW1 F R IY1 Z T U...,AY1 G OW1 T EY1 N V AW0 R IY1 Z T AA1 ...,100.000000,36.842105
19,L EH1 T S P L EY1 T AH0 G EH1 DH ER0 .,R EH1 D S P L AA1 Y AY1 K AH0 TH EH2 R,100.000000,45.000000
20,AY1 F IY1 L AE1 Z AY1 K AE1 N R IH0 ...,AY1 V IY1 EH1 L AE1 Z IH0 K AE1 N AH0 ...,100.000000,48.571429
21,Z IH1 R IY0 Z K AH1 M P AH0 N IY0 L IY1 S ...,AY1 S K AA1 M P AA1 N IH1 R IY0 Z IY1 ...,100.000000,45.312500
...,...,...,...,...
9987,T EH1 R AH0 S F AA1 R M IH0 NG IH1 Z W A...,T EH1 R AH0 K AE2 S F ER0 M AH0 G AO1 D IH0 ...,100.000000,37.500000
9990,T AA1 M W AA1 N T AH0 D T UW1 S IY1 W ...,D AA1 M W AH0 EH1 N T EH1 T T UW1 S AW...,100.000000,33.333333
9992,AY1 HH AE1 V T UW1 S T AA1 P DH IH1 S .,AY1 HH AE1 V T AA1 T S AA1 P T HH IH1 Z,80.000000,40.000000
9995,AY1 K AE1 N T D IH0 S K AH1 S IH1 T .,IY1 K AH0 D AE1 N T S K AH1 S IH0 T,100.000000,47.368421


In [17]:
cer_dataset.drop(['WER', 'CER'], axis=1, inplace=True)

print(cer_dataset)

                                               Sentence  \
9     T AA1 M   IH1 Z   P R AA1 B AH0 B L IY0   AH0 ...   
11    AY1 M   G OW1 IH0 NG   T UW1   F R IY1 Z   T U...   
19         L EH1 T S   P L EY1   T AH0 G EH1 DH ER0   .   
20    AY1   F IY1 L   AE1 Z   AY1   K AE1 N   R IH0 ...   
21    Z IH1 R IY0 Z   K AH1 M P AH0 N IY0   L IY1 S ...   
...                                                 ...   
9987  T EH1 R AH0 S   F AA1 R M IH0 NG   IH1 Z   W A...   
9990  T AA1 M   W AA1 N T AH0 D   T UW1   S IY1   W ...   
9992  AY1   HH AE1 V   T UW1   S T AA1 P   DH IH1 S   .   
9995      AY1   K AE1 N T   D IH0 S K AH1 S   IH1 T   .   
9996                 R IY1 M AH0   HH ER1 D   IH1 T   .   

                                         noisy_sentence  
9     T OW1 N AH0 S   B IY1 R   OW1 B AH0 B   R AY1 ...  
11    AY1   G OW1   T EY1 N V AW0   R IY1 Z   T AA1 ...  
19           R EH1 D S P   L AA1   Y AY1 K AH0 TH EH2 R  
20    AY1   V IY1   EH1 L   AE1 Z IH0 K   AE1 N AH0 ...  
2

/tmp/ipykernel_3587526/4099829987.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cer_dataset.drop(['WER', 'CER'], axis=1, inplace=True)


In [18]:
import re
cer_dataset['Sentence'] = cer_dataset['Sentence'].apply(lambda x: re.sub(r'\d', '', x))
cer_dataset['noisy_sentence'] = cer_dataset['noisy_sentence'].apply(lambda x: re.sub(r'\d', '', x))

print(cer_dataset)

                                               Sentence  \
9     T AA M   IH Z   P R AA B AH B L IY   AH   B IH...   
11    AY M   G OW IH NG   T UW   F R IY Z   T UW   D...   
19              L EH T S   P L EY   T AH G EH DH ER   .   
20    AY   F IY L   AE Z   AY   K AE N   R IH L AY  ...   
21    Z IH R IY Z   K AH M P AH N IY   L IY S T   AH...   
...                                                 ...   
9987  T EH R AH S   F AA R M IH NG   IH Z   W AY D L...   
9990  T AA M   W AA N T AH D   T UW   S IY   W AH T ...   
9992       AY   HH AE V   T UW   S T AA P   DH IH S   .   
9995           AY   K AE N T   D IH S K AH S   IH T   .   
9996                     R IY M AH   HH ER D   IH T   .   

                                         noisy_sentence  
9     T OW N AH S   B IY R   OW B AH B   R AY   AE B...  
11    AY   G OW   T EY N V AW   R IY Z   T AA D   IY...  
19                R EH D S P   L AA   Y AY K AH TH EH R  
20    AY   V IY   EH L   AE Z IH K   AE N AH L   EH ...  
2

/tmp/ipykernel_3587526/654431196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cer_dataset['Sentence'] = cer_dataset['Sentence'].apply(lambda x: re.sub(r'\d', '', x))
/tmp/ipykernel_3587526/654431196.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cer_dataset['noisy_sentence'] = cer_dataset['noisy_sentence'].apply(lambda x: re.sub(r'\d', '', x))


In [19]:
# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(cer_dataset, test_size=0.2, random_state=42)

In [20]:
# Save the training and testing sets to new CSV files
train_df.to_csv('data/train_sentences_noise_data.csv', index=False)
test_df.to_csv('data/test_sentences_noise_data.csv', index=False)

In [21]:
!pip install fsspec==2023.4.0

  Using cached fsspec-2023.4.0-py3-none-any.whl (153 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.10.0
    Uninstalling fsspec-2023.10.0:
      Successfully uninstalled fsspec-2023.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingface-hub 0.19.4 requires fsspec>=2023.5.0, but you have fsspec 2023.4.0 which is incompatible.


In [22]:
!pip install datasets -U

INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
  Using cached huggingface_hub-0.19.3-py3-none-any.whl.metadata (14 kB)
  Using cached huggingface_hub-0.19.2-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.19.1-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.19.0-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.18.0-py3-none-any.whl.metadata (13 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0


In [23]:
# Load the dataset from the JSON files
dataset = load_dataset('csv', data_files={'train': 'data/train_sentences_noise_data.csv',
                                           'test': 'data/test_sentences_noise_data.csv'}
                                           )

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

convert to json

In [24]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'noisy_sentence'],
        num_rows: 2228
    })
    test: Dataset({
        features: ['Sentence', 'noisy_sentence'],
        num_rows: 557
    })
})


In [25]:
pprint(dataset['train'][:5])
pprint(dataset['test'][:5])

{'Sentence': ['AE T   DH AH   K AA N V AH K EY SH AH N   ,   AO L   DH AH   F '
              'AE K AH L T IY   AH N D   D IY N Z   W AO R   DH EH R   D AA K '
              'T ER AH L   R IH G EY L Y AH   AH N D   AA K T AE G AH N AH L   '
              'HH AE T S   .',
              'DH AH   P R EH Z IH D AH N T S   K AA R   ER AY V D   AE T   DH '
              'AH   HH EH D K W AO R T ER Z   AH V   DH AH   N AE SH AH N AH '
              'L   IH N T EH L AH JH AH N S   S ER V AH S   .',
              'SH IY   S P IY K S   IH NG G L IH SH   .',
              'S OW SH AH L IH Z AH M   IH Z   DH AH   S EY M   AE Z   K AA M '
              'Y AH N IH Z AH M   ,   OW N L IY   B EH T ER   IH NG G L IH '
              'SH   .',
              'AY   S IY   AH   S ER JH   IH N   L OW N   G R OW TH   .'],
 'noisy_sentence': ['AH   T EH K OW   EH N F   AA K   EY D IY AH N AA R   TH '
                    'IY F AH K ER T IY   AE N D IY AH N Z   D AH B AH L Y UW   '
                    'AO R IH T

In [26]:
# Split the dataset into train and validation sets
train_dataset, val_dataset = train_test_split(dataset['train'], test_size=0.1, random_state=42)

In [27]:
from datasets import Dataset

train_data = Dataset.from_dict(train_dataset)  # Convert the train data to a dataset
val_data = Dataset.from_dict(val_dataset)      # Convert the validation data to a dataset

In [28]:
print(type(train_data))
print(type(val_data))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [29]:
# Check column names in train dataset
print(train_data.column_names)

# Check column names in validation dataset
print(val_data.column_names)

['Sentence', 'noisy_sentence']
['Sentence', 'noisy_sentence']


In [30]:
print(train_data['Sentence'][:5])
print(train_data['noisy_sentence'][:5])
#print(val_data[:5])

['N AA T   EH V R IY TH IH NG   IH M P AO R T AH N T   K AE N   B IY   K AW N T AH D   AH N D   N AA T   EH V R IY TH IH NG   DH AE T   K AE N   B IY   K AW N T AH D   IH Z   IH M P AO R T AH N T   .', 'T AA M   R IY D   TH ER D IY   B UH K S   L AE S T   Y IH R   .', 'F EY L Y ER Z   AA R   AO L W EY Z   F AA L OW D   B AY   S AH K S EH S AH Z   .', 'DH AH   S EH P ER AH T IH S T S   W AA N T   T UW   D IH S T EY B AH L AY Z   AE L JH IH R IY AH   .', 'AY   W AA N T   T UW   R EH N T   AH   S EY L B OW T   .']
['N OW T EH V ER   Y IH TH IH N   G IH M P AA L   T AE N   D K AE M   B IH K AW N   T EH T AA N D M OW   T EH V IH TH R IY   EH N T EY JH   HH AE T   K AE M B IH K   AW M   D EH T IY   S IY   P AA M ER D AE N D', 'T AA M ER   EH D TH ER T   AY B OW   OW G S R AE T S IY   AA R', 'EH F   EY L ER   EH Z   AE R IY AE L F ER S W AA N   OW D IY B S UW K IH K   EH S IH Z', 'AE T IH Z T S   W AA N T OW   T EH S D AE B   IH R IY Z IY AA L D IY   AH', 'IY W AH   EH M   T IY D IY   AO R AH

In [31]:
# Define the source and target language columns
source_lang = 'noisy_sentence'
target_lang = 'Sentence'

In [32]:
print(source_lang)

noisy_sentence


In [33]:
# Define the max_length for padding and truncation
max_length = 1024

In [34]:
# Initialize the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

# Tokenize the data
# The preprocess_function function is defined to preprocess the data by tokenizing the inputs and labels
def preprocess_function(examples):
    inputs = [f'{source_lang}: {text}' for text in examples[source_lang]]
    targets = examples[target_lang]
    encoding = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
    model_inputs = {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': tokenizer(targets, padding=True, truncation=True, return_tensors='pt')['input_ids'].squeeze()
    }
    return model_inputs

In [35]:
# Select a random data point from the train dataset
sample_data = train_data[0]

# Call the preprocess function on the sample data
processed_data = preprocess_function(sample_data)

# Inspect the output
print(processed_data)

{'input_ids': tensor([[    0,  2362, 26351,  ...,   234,     2,     1],
        [    0,  2362, 26351,  ...,  1437,  1437,     2],
        [    0,  2362, 26351,  ...,   384,     2,     1],
        ...,
        [    0,  2362, 26351,  ...,   234,     2,     1],
        [    0,  2362, 26351,  ...,  1437,  1437,     2],
        [    0,  2362, 26351,  ...,   211,     2,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0]]), 'labels': tensor([    0,   487, 14121,   255,  1437,  1437,   381,   725,   468,   248,
           38,   975,  8640,    38,   725, 36808,  1437,  1437,    38,   725,
          256,   221,    83,   673,   248,   255, 30227,   234,   255,  1437,
         1437,   229, 35722,   234,  1437,  1437,   163,    38,   975,  1437,
         1437,   229, 18463,   234,   255, 30227,   211,  1437, 

In [36]:
#train_data = preprocess_function(train_data)
#val_data = preprocess_function(val_data)

# Apply preprocess_function to train_data and val_data
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/2005 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

In [37]:
# # Access a few samples from train_data
# for i in range(5):
#     sample_input_ids = train_data['input_ids'][i]
#     sample_attention_mask = train_data['attention_mask'][i]
#     sample_labels = train_data['labels'][i]

#     print(f"Sample {i+1}:")
#     print("Input IDs:", sample_input_ids)
#     print("Attention Mask:", sample_attention_mask)
#     print("Labels:", sample_labels)
#     print()

In [38]:
# Prepare DataLoader for training and validation
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=16)

In [39]:
# define a data_collator function for batch processing
def data_collator(features):
    batch = {}
    # Pad input_ids and attention_mask to the maximum length within the batch
    max_length = max(len(feature['input_ids']) for feature in features)
    batch['input_ids'] = torch.stack([torch.tensor(feature['input_ids'] + [tokenizer.pad_token_id] * (max_length - len(feature['input_ids']))) for feature in features])
    batch['attention_mask'] = torch.stack([torch.tensor(feature['attention_mask'] + [0] * (max_length - len(feature['attention_mask']))) for feature in features])
    batch['labels'] = torch.stack([torch.tensor(feature['labels'] + [-100] * (max_length - len(feature['labels']))) for feature in features])
    return batch

In [40]:
# import accelerate
# import transformers

# transformers.__version__, accelerate.__version__

In [47]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="spell_correction_tatoeba_sentence",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [48]:
# verify passing the correct inputs to the trainer
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)
print("Tokenizer:", tokenizer)
print("Training Arguments:", training_args)

Train Dataset: Dataset({
    features: ['Sentence', 'noisy_sentence', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2005
})
Validation Dataset: Dataset({
    features: ['Sentence', 'noisy_sentence', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 223
})
Tokenizer: PreTrainedTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=3,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_f

In [49]:
torch.cuda.empty_cache()


In [50]:
# model is initialized with the BARTForConditionalGeneration class and moved to the GPU if available.
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

loading configuration file config.json from cache at /home/jinda/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "L

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [51]:
torch.cuda.empty_cache()

In [52]:
# The Seq2SeqTrainer is created with the defined model, training arguments, datasets, tokenizer, and data_collator
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Sentence, noisy_sentence. If Sentence, noisy_sentence are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2005
  Num Epochs = 30
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 10050


  0%|          | 0/10050 [00:00<?, ?it/s]

/home/jinda/anaconda3/envs/cs224n/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Sentence, noisy_sentence. If Sentence, noisy_sentence are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 223
  Batch size = 6


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.44535142183303833, 'eval_runtime': 7.0119, 'eval_samples_per_second': 31.803, 'eval_steps_per_second': 5.419, 'epoch': 1.0}


...

Model weights saved in spell_correction_tatoeba_sentence/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in spell_correction_tatoeba_sentence/checkpoint-10000/tokenizer_config.json
Special tokens file saved in spell_correction_tatoeba_sentence/checkpoint-10000/special_tokens_map.json
Deleting older checkpoint [spell_correction_tatoeba_sentence/checkpoint-8500] due to args.save_total_limit
/home/jinda/anaconda3/envs/cs224n/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Sentence, noisy_sentence. If Sentence, noisy_sentence are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.


  0%|          | 0/38 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.169845849275589, 'eval_runtime': 6.9174, 'eval_samples_per_second': 32.237, 'eval_steps_per_second': 5.493, 'epoch': 30.0}
{'train_runtime': 8379.9514, 'train_samples_per_second': 7.178, 'train_steps_per_second': 1.199, 'train_loss': 0.16116802483648804, 'epoch': 30.0}


TrainOutput(global_step=10050, training_loss=0.16116802483648804, metrics={'train_runtime': 8379.9514, 'train_samples_per_second': 7.178, 'train_steps_per_second': 1.199, 'train_loss': 0.16116802483648804, 'epoch': 30.0})

In [54]:
# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [55]:
test_dataset = dataset['test']

In [56]:
pprint(test_dataset['Sentence'][-10:])
pprint(test_dataset['noisy_sentence'][-10:])

['D OW N T   L EH T   Y AO R   P ER S IH N IH L   F IY L IH NG Z   G EH T   IH '
 'N   DH AH   W EY   AH V   Y AO R   W ER K   .',
 'DH EY   S EH D   DH AE T   DH EY   AA R AH N T   L AY K L IY   T UW   K R '
 'AY   .',
 'W EH R   D IH D   SH IY   G OW   L AE S T   W IY K EH N D   ?',
 'M AH Z L AH M Z   SH UH D   N AA T   AH B Y UW Z   HH OW M OW S EH K SH AH W '
 'AH L Z   IY V IH N   DH OW   DH EH R   D UW IH NG   S AH M TH IH NG   DH AE '
 'T S   N AA T   M AO R AH L IY   AE K S EH P T AH B AH L   IH N   IH S L AA '
 'M   .',
 'DH AE T   R UW M ER   M EY   B IY   T R UW   .',
 'HH AE V   Y UW   F IH N IH SH T   Y AO R   S K UW L   P R AA JH EH K T   Y '
 'EH T   ?',
 'F Y OW D ER   K L EY M D   HH IH Z   IH N AH S AH N S   .',
 'M AY   R IH L IH JH AH S   W UH M AH N   N EY B AW ER   F IY L Z   R IH L '
 'IY   IH N S AH K Y ER   W EH N   SH IY   S IY Z   EH N IY TH IH NG   AH B AW '
 'T   S AY AH N S   F IH K SH AH N   .',
 'S AE M IY   HH IH T   L EY L AH   OW V ER   DH AH   HH EH

In [57]:
print(len(test_dataset))

557


In [58]:
# Inspect column names
column_names = test_dataset.column_names
print("Column names:", column_names)

# Inspect column data types
for column in column_names:
    column_data = test_dataset[column]
    data_type = type(column_data[0]) if len(column_data) > 0 else "Unknown"
    print(f"Column name: {column}, Data type: {data_type}")

Column names: ['Sentence', 'noisy_sentence']
Column name: Sentence, Data type: <class 'str'>
Column name: noisy_sentence, Data type: <class 'str'>


In [59]:
# verification
# Test the model on speaker dataset
verify_test_dataset = dataset['test']
print(len(verify_test_dataset))
print((verify_test_dataset))

557
Dataset({
    features: ['Sentence', 'noisy_sentence'],
    num_rows: 557
})


In [60]:
test_data = test_dataset.map(preprocess_function, batched=True, batch_size=len(test_dataset))
model.eval()

Map:   0%|          | 0/557 [00:00<?, ? examples/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [61]:
predictions = []
references = []

for example in test_dataset:
    # This creates an input text by combining the values of the 'speaker' and 'prediction' fields from the current example.
    # It assumes that the example is a dictionary-like object with keys 'speaker' and 'prediction'.
    input_text = f"{example['noisy_sentence']}"

    # This creates a context where no gradients are computed, which can improve efficiency during inference.
    with torch.no_grad():

        # This tokenizes the input_text using the tokenizer and converts it into input IDs as a PyTorch tensor.
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # The generate method generates the output sequence based on the provided input.
        outputs = model.generate(input_ids=input_ids, max_length=max_length)

    predicted_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

    references.append(example['Sentence'])
    predictions.append(predicted_sentence)

In [62]:
# Verify that the number of predictions and references are the same
if len(predictions) == len(references):
    print("Number of predictions and references are the same.")
else:
    print("Mismatch in the number of predictions and references.")

Number of predictions and references are the same.


In [63]:
# Print the number of predictions and references
print("Number of predictions:", len(predictions))
print("Number of references:", len(references))

# print the length of the dataset
print("Number of rows in dataset:", len(test_dataset))

Number of predictions: 557
Number of references: 557
Number of rows in dataset: 557


In [64]:
for i in range(min(len(test_dataset['noisy_sentence']), len(predictions), len(references))):
    print(f"Sample {i+1}:")
    print("Test Dataset:", test_dataset['noisy_sentence'][i])
    print("Prediction:", predictions[i])
    print("Reference:", references[i])
    print("\n")

Sample 1:
Test Dataset: T IY   OW   S TH AE T   N EY M   R IH NG B OW   L EH L
Prediction: T AA M   DH AE T   M EH R IY   N OW   L EH L  .
Reference: D AH Z   DH AE T   N EY M   R IH NG   AH   B EH L   ?


Sample 2:
Test Dataset: S IY   L IY   IH S G   OY N D G OW   R IH N   AE SH UW S   T OW S T UW B IY D OW   L OW K IY
Prediction: Z IH R IY   IH Z   G OW IH NG   T UW   L AY N   AH   SH UH D   S T AA P AH L   K IY P  .
Reference: Z IH R IY   IH Z   G OW IH NG   T UW   R IY M AH Z   HH AW S   T UW   S T AH D IY   B AY AA L AH JH IY   .


Sample 3:
Test Dataset: EH M   AH   R AY   AH Z AY M AH N   AE N AH K S UW AH   T L AE M   N UW T AH D   AE V D ER   HH ER G R EY   AE T K AH S EH R AH S T ER Z   T IY EH S
Prediction: EH M EH R IY   AE Z   IH N   M AE N AH K S  ,   AH   T R AE M   N UW   DH AE T   F AO R   HH ER   G R EY D   K AH S EH L AH S T ER S T IH S 
Reference: M EH R IY   W AA Z   IH N   AE N   IH G Z AH L T AH N T   M UW D   AE F T ER   HH ER   G R EY T   IH G Z AE M   R IH Z 

In [65]:
from jiwer import wer, cer

# Assuming 'predictions' and 'references' are your sequences
# Calculate Word Error Rate (WER)
wer_value = wer(predictions, references)
wer_percentage = wer_value * 100
print(f"WER: {wer_percentage:.2f}%")

# Calculate Character Error Rate (CER)
cer_value = cer(predictions, references)
cer_percentage = cer_value * 100
print(f"CER: {cer_percentage:.2f}%")

WER: 61.93%
CER: 42.05%
